In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
from scipy.io import arff
import os

data_path = "/content/drive/MyDrive/NATOPS_sid_TRAIN.csv"
output_csv = "NATOPS_sid_TRAIN.csv"


In [ ]:
df = pd.read_csv(data_path)
df.head()

,isTest,fea1,fea2,fea3,fea4,fea5,fea6,fea7,fea8,fea9,...,fea17,fea18,fea19,fea20,fea21,fea22,fea23,fea24,sid,class
0,0,-0.372758,-1.821679,-0.846321,0.465208,-2.015072,-0.839242,-0.564097,-0.796225,-0.149604,...,-1.534954,-0.673190,-0.536343,-1.626957,-0.594337,0.619205,-1.771773,-0.810086,1,3
1,0,-0.367844,-1.841987,-0.846325,0.467033,-2.007557,-0.838151,-0.564499,-0.797622,-0.150012,...,-1.532795,-0.671919,-0.533816,-1.642514,-0.605328,0.617045,-1.796660,-0.818863,1,3
2,0,-0.378445,-1.821358,-0.839571,0.471135,-2.010042,-0.832021,-0.563753,-0.795704,-0.151608,...,-1.532478,-0.671555,-0.526319,-1.697145,-0.624302,0.624789,-1.738568,-0.788060,1,3
3,0,-0.386751,-1.845643,-0.848031,0.506153,-2.032552,-0.841696,-0.565008,-0.790238,-0.152350,...,-1.535441,-0.672198,-0.554538,-1.644413,-0.602884,0.634100,-1.749744,-0.816695,1,3
4,0,-0.417101,-1.941721,-0.885500,0.611207,-1.953282,-0.902529,-0.573550,-0.799730,-0.169575,...,-1.482552,-0.659393,-0.576196,-1.763092,-0.694843,0.680086,-1.664565,-0.857897,1,3


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
n_clusters = 10
cluster_ratios_df = pd.DataFrame()

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress warnings
warnings.filterwarnings("ignore")


In [ ]:
for sid in df['sid'].unique():
    # Subset the data for the current sample
    sample_data = df[df['sid'] == sid]

    # Extract features for clustering
    features = sample_data.drop(['sid', 'class','isTest'], axis=1)

    # Standardize features
    scaler = StandardScaler()
    features_standardized = scaler.fit_transform(features)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    sample_data['cluster'] = kmeans.fit_predict(features_standardized)

    # Calculate cluster ratios for the current sample
    cluster_ratios = sample_data.groupby('cluster').size() / len(sample_data)

    # Append the cluster ratios to the new DataFrame
    cluster_ratios['sid'] = sid
    cluster_ratios_df = cluster_ratios_df.append(cluster_ratios, ignore_index=True)

# Merge cluster ratios with the original data
df = pd.merge(df[['sid', 'class']], cluster_ratios_df, on='sid')

# Display the final DataFrame


In [ ]:
df.head()

,sid,class,0,1,2,3,4,5,6,7,8,9
0,1,3,0.078431,0.313725,0.196078,0.156863,0.039216,0.058824,0.019608,0.039216,0.039216,0.058824
1,1,3,0.078431,0.313725,0.196078,0.156863,0.039216,0.058824,0.019608,0.039216,0.039216,0.058824
2,1,3,0.078431,0.313725,0.196078,0.156863,0.039216,0.058824,0.019608,0.039216,0.039216,0.058824
3,1,3,0.078431,0.313725,0.196078,0.156863,0.039216,0.058824,0.019608,0.039216,0.039216,0.058824
4,1,3,0.078431,0.313725,0.196078,0.156863,0.039216,0.058824,0.019608,0.039216,0.039216,0.058824


In [ ]:
df.shape

(18360, 12)

In [ ]:
cluster_ratios_df.shape

(360, 11)